In [16]:
import cv2
import numpy as np

# Function to display an image
def display_image(image, title):
    if image is not None and image.size > 0:
        cv2.imshow(title, image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("Error: Invalid image or size")

# Read the input image
image = cv2.imread("ISIC_0024320.jpg")

# Display the original image
display_image(image, "Original Image")

if image is not None and image.size > 0:
    # Create a mask initialized with zeros
    mask = np.zeros(image.shape[:2], dtype=np.uint8)

    # Initialize the rectangle for GrabCut
    rect = (50, 50, image.shape[1]-50, image.shape[0]-50)

    # Draw the rectangle on a copy of the original image
    image_with_rectangle = image.copy()
    cv2.rectangle(image_with_rectangle, (50, 50), (image.shape[1]-50, image.shape[0]-50), (0, 255, 0), 3)
    display_image(image_with_rectangle, "Image with Rectangle")

    # Initialize the foreground and background models
    bgdModel = np.zeros((1, 65), np.float64)
    fgdModel = np.zeros((1, 65), np.float64)

    # Apply GrabCut segmentation
    cv2.grabCut(image, mask, rect, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_RECT)

    # Modify the mask to make all non-zero pixels (sure foreground and likely foreground) to be 1
    mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')

    # Apply the modified mask to the input image
    segmented_image = image * mask2[:, :, np.newaxis]

    # Display the segmented image
    display_image(segmented_image, "Segmented Image")


In [10]:
import cv2
import numpy as np

# Function to display an image
def display_image(image, title):
    if image is not None and image.size > 0:
        cv2.imshow(title, image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("Error: Invalid image or size")

# Read the input image
img = cv2.imread("ISIC_0024320.jpg")

if img is not None:
# Resize image to the specified width and height
    width = 512
    height = 512
    img = cv2.resize(img, (width, height))
            
# Convert RGB image to grayscale
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
    # Apply Morphological Black-Hat transformation
    kernel_size = 15  # Adjust kernel size as needed
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_size, kernel_size))
    blackhat_image = cv2.morphologyEx(gray_image, cv2.MORPH_BLACKHAT, kernel)
                
# Create a mask for inpainting task
    _, mask = cv2.threshold(blackhat_image, 10, 255, cv2.THRESH_BINARY)
    mask = cv2.resize(mask, (img.shape[1], img.shape[0]))
           
# Apply inpainting algorithm on the original image using the mask
    img = cv2.inpaint(img, mask, inpaintRadius=3, flags=cv2.INPAINT_TELEA)

# Save the modified image, overwriting the original file
    cv2.imwrite("ISIC_0024320.jpg", img)
            
else:
    print("Error reading image:", image_path)

# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply Gaussian blur to reduce noise
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# Perform thresholding to segment the lesion
_, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Perform morphological operations to improve segmentation
kernel = np.ones((5,5), np.uint8)
opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)
sure_bg = cv2.dilate(opening, kernel, iterations=3)

# Find sure foreground area using distance transform
dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
_, sure_fg = cv2.threshold(dist_transform, 0.7*dist_transform.max(), 255, 0)

# Find unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg, sure_fg)

# Marker labelling
_, markers = cv2.connectedComponents(sure_fg)

# Add one to all labels so that sure background is not 0, but 1
markers = markers + 1

# Mark unknown region with 0
markers[unknown == 255] = 0

# Apply watershed algorithm
markers = cv2.watershed(img, markers)
img[markers == -1] = [0, 255, 0]  # Mark boundary region in blue

# Display the segmented image
display_image(img, "Segmented Image")


## Segmentation method 2

In [42]:
import cv2
import numpy as np

# Function to display an image
def display_image(image, title):
    if image is not None and image.size > 0:
        cv2.imshow(title, image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("Error: Invalid image or size")

# Read the input image
image = cv2.imread("ISIC_0024320.jpg")

# Display the original image
display_image(image, "Original Image")

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Display the grayscale image
display_image(gray, "Grayscale Image")

# Apply Gaussian blur to reduce noise
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# Display the blurred image
display_image(blurred, "Blurred Image")

# Perform thresholding to obtain binary image
_, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Display the thresholded image
display_image(thresh, "Thresholded Image")

# Find contours in the binary image
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Get the largest contour (assumed to be the lesion)
largest_contour = max(contours, key=cv2.contourArea)

# Create a mask for the lesion area
# lesion_mask = cv2.threshold(blackhat_image, 10, 255, cv2.THRESH_BINARY)
lesion_mask = np.zeros_like(gray)
cv2.drawContours(lesion_mask, [largest_contour], -1, 255, cv2.FILLED)

# Display the lesion mask
display_image(lesion_mask, "Lesion Mask")

# Apply the mask to create a black image with only the lesion area
lesion_only = cv2.bitwise_and(image, image, mask=lesion_mask)

# Display the black image with only the lesion area
display_image(lesion_only, "Lesion Only")


In [49]:
import cv2
import numpy as np

# Function to display an image
def display_image(image, title):
    if image is not None and image.size > 0:
        cv2.imshow(title, image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("Error: Invalid image or size")

# Read the input image
image = cv2.imread("ISIC_0024320.jpg")

# Display the original image
display_image(image, "Original Image")

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Display the grayscale image
display_image(gray, "Grayscale Image")

# Apply Gaussian blur to reduce noise
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# Display the blurred image
display_image(blurred, "Blurred Image")

# Perform thresholding to obtain binary image
_, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Display the thresholded image
display_image(thresh, "Thresholded Image")

# Find contours in the binary image
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


# Get the largest contour (assumed to be the lesion)
largest_contour = max(contours, key=cv2.contourArea)

# Create a mask for the lesion area
lesion_mask = np.zeros_like(gray)
cv2.drawContours(lesion_mask, [largest_contour], -1, 255, cv2.FILLED)

# Apply the mask to create a black image with only the lesion area
lesion_only = cv2.bitwise_and(image, image, mask=lesion_mask)

# Display the black image with only the lesion area
display_image(lesion_only, "Lesion Only")
